In [1]:
import json

In [11]:
import base64
from algosdk.v2client import algod
from algosdk import account, mnemonic
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.transaction import *

# Accounts

In [4]:
mnemonic1 = "kiss hope rubber day sphere identify journey material just depend case camp wash like simple dash push tone truck change chicken peace wrong able glove"
mnemonic2 = "soup window nice vapor chimney also sugar cram present dry blame obscure illegal include scheme flame gaze twist crop confirm vault economy hip abandon favorite"
mnemonic3 = "note mix true cream demise inside bench ozone cinnamon frame charge crush into travel ensure blanket rose volume blade humor truly final park abstract boat"

In [5]:
# For ease of reference, add account public and private keys to
# an accounts dict.
accounts = {}
counter = 1
for m in [mnemonic1, mnemonic2, mnemonic3]:
    accounts[counter] = {}
    accounts[counter]['pk'] = mnemonic.to_public_key(m)
    accounts[counter]['sk'] = mnemonic.to_private_key(m)
    counter += 1

AttributeError: module 'algosdk.mnemonic' has no attribute 'to_public_key'

# Connect to Client

In [6]:
# Specify the node address and token.

algod_address = "http://localhost:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"

# Initialize an algod client
algod_client = algod.AlgodClient(algod_token=algod_token, algod_address=algod_address)

In [7]:
print("Account 1 address: {}".format(accounts[1]['pk']))
print("Account 2 address: {}".format(accounts[2]['pk']))

KeyError: 'pk'

# Create Asset (Minting)

* Account 1 creates an asset called latinum and
* sets Account 2 as the manager, reserve, freeze, and clawback address.

In [9]:
# Get network params for transactions before every transaction.
params = algod_client.suggested_params()

# Account 1 creates an asset called latinum and
# sets Account 2 as the manager, reserve, freeze, and clawback address.
# Asset Creation transaction

txn = AssetConfigTxn(
    sender=accounts[1]['pk'],
    sp=params,
    total=1000,
    default_frozen=False,
    unit_name="LATINUM",                                                    
    asset_name="latinum",
    manager=accounts[1]['pk'],
                                                                       
    freeze=accounts[1]['pk'],
    clawback=accounts[1]['pk'],
    url="https://path/to/my/asset/details", 
    decimals=0)
    
# Sign with secret key of creator
stxn = txn.sign(accounts[1]['sk'])


In [10]:
# Send the transaction to the network and retrieve the txid.
try:
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4)  
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))
   
except Exception as err:
    print(err)
# Retrieve the asset ID of the newly created asset by first
# ensuring that the creation transaction was confirmed,
# then grabbing the asset id from the transaction.

Signed transaction with txID: ZGYNZRHP2BTCJOCYJKVWAQIDAF2LCKLYAPWOQ4SST3ALUWXUZRAQ
TXID:  ZGYNZRHP2BTCJOCYJKVWAQIDAF2LCKLYAPWOQ4SST3ALUWXUZRAQ
Result confirmed in round: 2


In [11]:
print("Transaction information: {}".format(
    json.dumps(confirmed_txn, indent=4)))

Transaction information: {
    "asset-index": 1002,
    "confirmed-round": 2,
    "pool-error": "",
    "txn": {
        "sig": "3RjUXHhpClMy+J2oQXgTLuc01bSat09WzML9DTaj0CxIsCo9pP5LQpEIekq6jxxJzTe1kL1fXJKFRPkPbNLnDg==",
        "txn": {
            "apar": {
                "an": "latinum",
                "au": "https://path/to/my/asset/details",
                "c": "FTHK4OKIVNDJWLGFJZPO25XYETWJWFDUYPROWHZ6LHHNELUNEWWNP6RMHI",
                "f": "FTHK4OKIVNDJWLGFJZPO25XYETWJWFDUYPROWHZ6LHHNELUNEWWNP6RMHI",
                "m": "FTHK4OKIVNDJWLGFJZPO25XYETWJWFDUYPROWHZ6LHHNELUNEWWNP6RMHI",
                "r": "FTHK4OKIVNDJWLGFJZPO25XYETWJWFDUYPROWHZ6LHHNELUNEWWNP6RMHI",
                "t": 1000,
                "un": "LATINUM"
            },
            "fee": 1000,
            "fv": 1,
            "gen": "sandnet-v1",
            "gh": "MlInopO7QUmtjVCotTfxhehLOqAnFdg6molUtA5nxi0=",
            "lv": 1001,
            "snd": "FTHK4OKIVNDJWLGFJZPO25XYETWJWFDUYPROWHZ6LHHNELUNEWWNP6R

In [17]:
#   Utility function used to print created asset for account and assetid
def print_created_asset(algodclient, account, assetid):    
    # note: if you have an indexer instance available it is easier to just use this
    # response = myindexer.accounts(asset_id = assetid)
    # then use 'account_info['created-assets'][0] to get info on the created asset
    account_info = algodclient.account_info(account)
    idx = 0;
    for my_account_info in account_info['created-assets']:
        scrutinized_asset = account_info['created-assets'][idx]
        idx = idx + 1       
        if (scrutinized_asset['index'] == assetid):
            print("Asset ID: {}".format(scrutinized_asset['index']))
            print(json.dumps(my_account_info['params'], indent=4))
            break

#   Utility function used to print asset holding for account and assetid
def print_asset_holding(algodclient, account, assetid):
    # note: if you have an indexer instance available it is easier to just use this
    # response = myindexer.accounts(asset_id = assetid)
    # then loop thru the accounts returned and match the account you are looking for
    account_info = algodclient.account_info(account)
    idx = 0
    for my_account_info in account_info['assets']:
        scrutinized_asset = account_info['assets'][idx]
        idx = idx + 1        
        if (scrutinized_asset['asset-id'] == assetid):
            print("Asset ID: {}".format(scrutinized_asset['asset-id']))
            print(json.dumps(scrutinized_asset, indent=4))
            break


In [18]:
try:
    # Pull account info for the creator
    # account_info = algod_client.account_info(accounts[1]['pk'])
    # get asset_id from tx
    # Get the new asset's information from the creator account
    ptx = algod_client.pending_transaction_info(txid)
    asset_id = ptx["asset-index"]
    print_created_asset(algod_client, accounts[1]['pk'], asset_id)
    print_asset_holding(algod_client, accounts[1]['pk'], asset_id)
except Exception as e:
    print(e)

Asset ID: 1002
{
    "clawback": "FTHK4OKIVNDJWLGFJZPO25XYETWJWFDUYPROWHZ6LHHNELUNEWWNP6RMHI",
    "creator": "FTHK4OKIVNDJWLGFJZPO25XYETWJWFDUYPROWHZ6LHHNELUNEWWNP6RMHI",
    "decimals": 0,
    "default-frozen": false,
    "freeze": "FTHK4OKIVNDJWLGFJZPO25XYETWJWFDUYPROWHZ6LHHNELUNEWWNP6RMHI",
    "manager": "FTHK4OKIVNDJWLGFJZPO25XYETWJWFDUYPROWHZ6LHHNELUNEWWNP6RMHI",
    "name": "latinum",
    "name-b64": "bGF0aW51bQ==",
    "reserve": "FTHK4OKIVNDJWLGFJZPO25XYETWJWFDUYPROWHZ6LHHNELUNEWWNP6RMHI",
    "total": 1000,
    "unit-name": "LATINUM",
    "unit-name-b64": "TEFUSU5VTQ==",
    "url": "https://path/to/my/asset/details",
    "url-b64": "aHR0cHM6Ly9wYXRoL3RvL215L2Fzc2V0L2RldGFpbHM="
}
Asset ID: 1002
{
    "amount": 1000,
    "asset-id": 1002,
    "is-frozen": false
}


# Change the Manager

The current manager(Account 2) issues an asset configuration transaction that assigns Account 1 as the new manager.

In [27]:
# Keep reserve, freeze, and clawback address same as before, i.e. account 2
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
# params.fee = 1000
# params.flat_fee = True

# asset_id = 328952;

txn = AssetConfigTxn(
    sender=accounts[1]['pk'],
    sp=params,
    index=asset_id, 
    manager=accounts[3]['pk'],
    reserve=accounts[1]['pk'],
    freeze=accounts[1]['pk'],
    clawback=accounts[1]['pk'])
# sign by the current manager - Account 2
stxn = txn.sign(accounts[1]['sk'])
# txid = algod_client.send_transaction(stxn)
# print(txid)

In [28]:
# Wait for the transaction to be confirmed
# Send the transaction to the network and retrieve the txid.
try:
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4) 
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))
 
except Exception as err:
    print(err)

Signed transaction with txID: UGAG7ARQ3XPWAVVMERIQGGVNSI6R24PB6I6UJCEONR5GTMSYVJFA
TXID:  UGAG7ARQ3XPWAVVMERIQGGVNSI6R24PB6I6UJCEONR5GTMSYVJFA
Result confirmed in round: 7


In [30]:
# Check asset info to view change in management. manager should now be account 1
print_created_asset(algod_client, accounts[3]['pk'], asset_id)

# Opt-in

In [20]:
# OPT-IN

# Check if asset_id is in account 3's asset holdings prior
# to opt-in
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
# params.fee = 1000
# params.flat_fee = True

account_info = algod_client.account_info(accounts[2]['pk'])
holding = None
idx = 0
for my_account_info in account_info['assets']:
    scrutinized_asset = account_info['assets'][idx]
    idx = idx + 1    
    if (scrutinized_asset['asset-id'] == asset_id):
        holding = True
        break

if not holding:

    # Use the AssetTransferTxn class to transfer assets and opt-in
    txn = AssetTransferTxn(
        sender=accounts[2]['pk'],
        sp=params,
        receiver=accounts[2]["pk"],
        amt=0,
        index=asset_id)
    stxn = txn.sign(accounts[2]['sk'])
    # Send the transaction to the network and retrieve the txid.
    try:
        txid = algod_client.send_transaction(stxn)
        print("Signed transaction with txID: {}".format(txid))
        # Wait for the transaction to be confirmed
        confirmed_txn = wait_for_confirmation(algod_client, txid, 4) 
        print("TXID: ", txid)
        print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))
 
    except Exception as err:
        print(err)
    # Now check the asset holding for that account.
    # This should now show a holding with a balance of 0.
    print_asset_holding(algod_client, accounts[3]['pk'], asset_id)


# Asset Transfer

In [22]:
# transfer asset of 10 from account 1 to account 3
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
# params.fee = 1000
# params.flat_fee = True
txn = AssetTransferTxn(
    sender=accounts[1]['pk'],
    sp=params,
    receiver=accounts[2]["pk"],
    amt=100,
    index=asset_id)
stxn = txn.sign(accounts[1]['sk'])
# Send the transaction to the network and retrieve the txid.
try:
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4) 
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))

except Exception as err:
    print(err)
# The balance should now be 10.
print_asset_holding(algod_client, accounts[2]['pk'], asset_id)


Signed transaction with txID: KSLWG7KWC3LAF54XS3VRVAGNDQEKB5UVS32BKRT6SKVQMFAY3SPA
TXID:  KSLWG7KWC3LAF54XS3VRVAGNDQEKB5UVS32BKRT6SKVQMFAY3SPA
Result confirmed in round: 5
Asset ID: 1002
{
    "amount": 200,
    "asset-id": 1002,
    "is-frozen": false
}


# Asset Freeze

In [31]:
params = algod_client.suggested_params()

# The freeze address (Account 2) freezes Account 3's latinum holdings.

txn = AssetFreezeTxn(
    sender=accounts[1]['pk'],
    sp=params,
    index=asset_id,
    target=accounts[2]["pk"],
    new_freeze_state=True   
    )
stxn = txn.sign(accounts[1]['sk'])
# Send the transaction to the network and retrieve the txid.
try:
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4)  
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))    
except Exception as err:
    print(err)
# The balance should now be 10 with frozen set to true.
print_asset_holding(algod_client, accounts[3]['pk'], asset_id)

Signed transaction with txID: IEUTUBXS5JWOU6EUWOPO77ZGTSBBJEXCAHUU6T3WLK7MK2RGONDA


TXID:  IEUTUBXS5JWOU6EUWOPO77ZGTSBBJEXCAHUU6T3WLK7MK2RGONDA
Result confirmed in round: 8


# Revoke Asset

In [37]:
# The clawback address (Account 2) revokes 10 latinum from Account 3 and places it back with Account 1.
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
# params.fee = 1000
# params.flat_fee = True

# Must be signed by the account that is the Asset's clawback address
txn = AssetTransferTxn(
    sender=accounts[1]['pk'],
    sp=params,
    receiver=accounts[1]["pk"],
    amt=100,
    index=asset_id,
    revocation_target=accounts[2]['pk']
    )
stxn = txn.sign(accounts[1]['sk'])
# Send the transaction to the network and retrieve the txid.
try:
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4)
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))      
except Exception as err:
    print(err)
# The balance of account 2 should now be 0.
# account_info = algod_client.account_info(accounts[2]['pk'])
print("Account 2")
print_asset_holding(algod_client, accounts[2]['pk'], asset_id)

# The balance of account 1 should increase by 200 to 1000.
print("Account 1")
print_asset_holding(algod_client, accounts[1]['pk'], asset_id)

Signed transaction with txID: TVP7XPVKHTIMMVDO3TNMHO6SCXMA2QZDQAJACXQQSG24JA5LAUTQ
TXID:  TVP7XPVKHTIMMVDO3TNMHO6SCXMA2QZDQAJACXQQSG24JA5LAUTQ
Result confirmed in round: 10
Account 2
Asset ID: 1002
{
    "amount": 0,
    "asset-id": 1002,
    "is-frozen": true
}
Account 1
Asset ID: 1002
{
    "amount": 1000,
    "asset-id": 1002,
    "is-frozen": false
}


# Destroy Asset

In [38]:
# With all assets back in the creator's account,
# the manager (Account 3) destroys the asset.
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
# params.fee = 1000
# params.flat_fee = True

# Asset destroy transaction
txn = AssetConfigTxn(
    sender=accounts[3]['pk'],
    sp=params,
    index=asset_id,
    strict_empty_address_check=False
    )

# Sign with secret key of creator
stxn = txn.sign(accounts[3]['sk'])
# Send the transaction to the network and retrieve the txid.
# Send the transaction to the network and retrieve the txid.
try:
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4) 
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))     
except Exception as err:
    print(err)

# Asset was deleted.
try:
    print("Account 2 must do a transaction for an amount of 0, " )
    print("with a close_assets_to to the creator account, to clear it from its accountholdings")
    print("For Account 1, nothing should print after this as the asset is destroyed on the creator account")
   
    print_asset_holding(algod_client, accounts[1]['pk'], asset_id)
    print_created_asset(algod_client, accounts[1]['pk'], asset_id)
    # asset_info = algod_client.asset_info(asset_id)
except Exception as e:
    print(e)

Signed transaction with txID: TWG45OIJ4NSU3UYUITQ5NR77T2RKW2OCHTDCX3I7BLVZYNPP7HPA
TXID:  TWG45OIJ4NSU3UYUITQ5NR77T2RKW2OCHTDCX3I7BLVZYNPP7HPA
Result confirmed in round: 11
Account 2 must do a transaction for an amount of 0, 
with a close_assets_to to the creator account, to clear it from its accountholdings
For Account 1, nothing should print after this as the asset is destroyed on the creator account
